In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib
import seaborn
import pandas
import pickle
import numpy
import datetime
import math
import os
import scipy


In [2]:
import sys
import datetime

import simulation
import data_handler
import environment
import agent 

import imp
imp.reload(agent)
imp.reload(simulation)
imp.reload(data_handler)
imp.reload(environment)

<module 'environment' from '/home/ignavermeulen/modeling_charging_behavior/environment.py'>

In [11]:
%time sim_phev_to_high = simulation.Simulation("data/input_parameters/parameters.json", \
    overwrite_parameters = {'agent_initialization': 'load_and_use', \
    'filepath_agent_database': 'data/agent_database/all_non_changing_agents/', 'number_of_agents': 10, \
    'transform_parameters': { \
            "prob_no_transform": 0.0, \
            "prob_to_low_fev": 0.0, \
            "prob_to_high_fev": 1.0 \
          }, \
    'agent_creation_method': 'random', \
    'skip_high_fev_agents': True, \
    'skip_low_fev_agents': True})

INFO: Initialization
	INFO: Loading sessions took 1.79 seconds
	INFO: Detailed preprocessing took 2.49 seconds
	INFO: Initializing (load_and_use) 10 random agents
		INFO: 10 of 10 agents are initialized
			INFO: 10 of 2287 users are processed
		INFO: Initialized 10 agents in 18.35 seconds
	INFO: Initialization took 20.95 seconds
CPU times: user 20.3 s, sys: 824 ms, total: 21.2 s
Wall time: 21.3 s


In [ ]:
%time sim_phev_to_high.repeat_simulation(repeat = 1, method = 'relMAE', measures = []) 

In [6]:
a = list(sim_phev_to_high.agents.values())[0]

In [8]:
len(a.all_simulated_sessions[0])

272

In [6]:
all_sessions = pandas.DataFrame()
for a in sim_phev_to_high.agents.values():
    all_sessions = all_sessions.append(a.all_simulated_sessions[0])

In [7]:
len(all_sessions)

35680

In [8]:
path_to_file = 'data/sessions/ChargeSessions_simulated_phev_to_high.pkl'
with open(path_to_file, 'wb') as data_file:
    pickle.dump(all_sessions, data_file)

In [12]:
filepath_agent_database = 'data/agent_database/phev_to_high_transformed_agents_test/'
for agent in sim_phev_to_high.agents.values():
    my_agent_data = agent._get_agent_data()
    with open(filepath_agent_database + agent.ID + '.pkl', 'wb') as agent_file:
        pickle.dump(my_agent_data, agent_file)

In [ ]:
for sim_repeat in range(5):
    for agent in sim_phev_to_high.values():
        agent.all_simulated_sessions

In [2]:
def save_agent(directory, agent_data, agent_ID):
    with open(directory + agent_ID + '.pkl', 'wb') as agent_file:
        pickle.dump(agent_data, agent_file)
        
def load_agent(directory, agent_ID):
    with open(directory + agent_ID + '.pkl', 'rb') as agent_file:
        agent_data = pickle.load(agent_file)
    return agent_data

def load_dict_agents_data(directory, skip_changing_users = True):
    result = {}
    all_agent_IDs = set([files[:-4] for files in os.listdir(directory) if files[0] != '.'])
    for ID in all_agent_IDs:
        if skip_changing_users and (ID in high_to_low_users or ID in low_to_high_users):
            continue
        agent_data = load_agent(directory, ID)
        if agent_data['user_type'] == 'regulier':
            result[ID] = agent_data
    return result

def get_dataframe_data(data_agents):
    stripped_dict = {}
    for agent in data_agents:
        stripped_dict[agent] = {}
        for key, val in data_agents[agent].items():
            if (isinstance(val, int) or isinstance(val, float) or isinstance(val, str)):
                stripped_dict[agent][key] = val
    return pandas.DataFrame.from_dict(stripped_dict, orient = 'index')

def load_agent_sessions(directory, data = pandas.DataFrame(), skip_changing_users = True):
    if len(data) == 0:
        path_to_file = 'data/sessions/ChargeSessions_general_filtered.pkl'
        with open(path_to_file, 'rb') as data_file:
            data = pickle.load(data_file)
    result = {}
    all_agent_IDs = set([files[:-4] for files in os.listdir(directory) if files[0] != '.'])
    for i, ID in enumerate(all_agent_IDs):
        if (i % 200 == 0):
            print("Loaded sessions for %d of %d agents" %(i, len(all_agent_IDs)))
        if skip_changing_users and (ID in high_to_low_users or ID in low_to_high_users):
            continue
        df = data.loc[data['ID'] == ID]
        df = df.sort_values(by = 'start_connection', inplace = False)
        result[ID] = df
    return result

def update_general_data():
    path_to_file = 'data/sessions/ChargeSessions_general.pkl'
    with open(path_to_file, 'rb') as data_file:
        data = pickle.load(data_file)
    df['connection_duration'] = df.apply(lambda row: row.end_connection - row.start_connection, axis = 1)
    df = df[df.connection_duration > min_duration_session] 
    path_to_file = 'data/sessions/ChargeSessions_general_filtered.pkl'
    with open(path_to_file, 'wb') as data_file:
            pickle.dump(df, data_file)

def update_agent_data(agent_dir, agent_ID, data_key, data, in_place = False, agent_data = None):
    ''' Updates the given agent with the given data in the database. The key data_key is added to the agent dictionary,
        and the data is added as value. This agent is then again saved in the same spot. 
    '''
    if agent_data == None:
        agent_data = load_agent(agent_dir, agent_ID)
    if not in_place:
        agent_data[data_key] = data
    save_agent(agent_dir, agent_data, agent_ID)

In [3]:
high_to_low_users = ['5D763B2DCBFD274859F43EE1456A1DD0B68F72A82E0A8564372C4BD46B95DAC8',
                    '8DD214E9F9611E7D01E662E940AC973BCC34CF45C3D43D3306DFD3F9AEB5075C',
                    '9A79FD1E5649B3FA3978C6B7E682571ED5570F5D1A35CDE4E80C6F30DFCFAF2B',
                    '09CAA3715AE1BC35FC152487BBD693275E97DE039207824D1063E5A7A0E2DD6E',
                    '144984E1475253AC45D743910C4CF344A2D315EA12ACD21AB58971105DEAF522',
                    'B8256507B35A603D43CF7437CCC5712F33F1F54B82EB416FEA942725AAD3831C',
                    'F32DBA25D91138963F85144783F71A16E4C2CF7D1523F0644A6404B4C2F227C4',
                    'F9541CDC30F3E36CEE3EE6954D0AC7D7925C5539D5E751E79AD13734E455ADBC']
low_to_high_users = ['6D47DA110FAA422E77B8DA82975F644315AC05C3D0E38B12A80816793452D857',
                    '8C45CDFC57F283C9BFB268CBD39725923FF8D22BCDF75E3E4FD054B079872459',
                    '9C510710140B57F6EB58CC760CCAE70064D161D0B993741CC479FAF65175B2CB',
                    '9D2D9AD61D9BCBF8A3CBBBEE6A031A7152EF2C28235E54CC5E1A478CC9BC9F73',
                    '17AFFF335DB8BF72259DD11C23D375ECA607613B2D9E1379AFF0922888A8EE90',
                    '53DDDDFF98D9CB22915B054A55F663BFDE941FFD8039A03248E559945C096031',
                    '80E9884E3E4B152B66E2B3A2861FE89BE1D48A346C16EF8739F9714F4BD6AEBB',
                    '606BD9C4F45EF4BD2710D2C4B178BF7356D2D9437BDF53B5593AB87F35A061B1',
                    '17897EE886276B2EA379234AC9112C0816C0C917977A8DB4E3282A3C9BE7D018',
                    '918130F5F1FC762D62308F082000F698F77C6CDBD34F9F4915835D484663A509',
                    'EDA3B229C2661C6E479A565173F553DB7A10275D5360E542DA626F96832C12C6',
                    'F346C8B51850F3D811CCDE0EC612153028C9F69E9B79C1FA9E4F5FA75F470AA9',
                    'FDC8791D2EE934F8A1DD22816E149DD2E37E185C985C772D38D44FE74AA207DE',
                    'F20D73D834CB2523C7E8E55643E2A7E6467DA5D4B4EBCA3547BC9FE49009E731']
wth_are_u_doing_users = ['7DA773D56D9BABA6B82139F286FA593EB0685317075EE743F05A3DDE1A1A552A',
                      '62ABF94F3B61963B34B78EA7F062DE8681015F17730270CCAA8F6BD61828889F',
                      '70E685BA7C2BC723DA307881476A3E5FC3AD1FCABDDFBF3A5BBA69E8BBB53B94']

In [4]:
% time data_agents = load_dict_agents_data('data/agent_database/all_non_changing_agents/')
data_agents_df = get_dataframe_data(data_agents)

CPU times: user 51.5 s, sys: 1.57 s, total: 53.1 s
Wall time: 1min 5s


In [5]:
path_to_file = 'data/sessions/ChargeSessions_general_filtered.pkl'
with open(path_to_file, 'rb') as data_file:
    general_data = pickle.load(data_file)

In [6]:
with open('data/sessions/ChargeSessions_general_expanded_per_agent.pkl', 'rb') as df:
    sessions_data_expanded = pickle.load(df)
    
dfs_phev = []
dfs_low_fev = []
dfs_high_fev = []

for i, a in enumerate(sessions_data_expanded):
    if a not in data_agents:
        continue
    if data_agents[a]['battery_category'] == 'phev':
        dfs_phev.append(sessions_data_expanded[a])
    elif data_agents[a]['battery_category'] == 'fev_low':
        dfs_low_fev.append(sessions_data_expanded[a])
    elif data_agents[a]['battery_category'] == 'fev_high':
        dfs_high_fev.append(sessions_data_expanded[a])


phev_sessions = pandas.concat(dfs_phev)
low_fev_sessions = pandas.concat(dfs_low_fev)
high_fev_sessions = pandas.concat(dfs_high_fev)

phev_sessions['car_type']  = ['phev'] * len(phev_sessions)
low_fev_sessions['car_type']  = ['low_fev'] * len(low_fev_sessions)
high_fev_sessions['car_type']  = ['high_fev'] * len(high_fev_sessions)

all_sessions = pandas.concat([phev_sessions, low_fev_sessions, high_fev_sessions])

In [7]:
temp_agent_sessions = {}
prev = datetime.datetime.now()
for i, a in enumerate(data_agents):
    if i % 100 == 0:
        print("%d of %d, took: %s" %(i, len(data_agents), datetime.datetime.now() - prev))
        prev = datetime.datetime.now()
    df = all_sessions.loc[all_sessions.ID == a]
    df['previous_location_key'] = df.location_key.shift(1)
    df = df[df.previous_location_key != 'NaT']
    temp_agent_sessions[a] = df

0 of 2242, took: 0:00:00.000726


/usr/local/lib/python3.4/dist-packages/ipykernel/__main__.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


100 of 2242, took: 0:01:55.249061
200 of 2242, took: 0:01:46.285261
300 of 2242, took: 0:01:54.933410
400 of 2242, took: 0:01:53.219279
500 of 2242, took: 0:01:51.797713
600 of 2242, took: 0:01:50.956315
700 of 2242, took: 0:01:53.129710
800 of 2242, took: 0:01:55.880877
900 of 2242, took: 0:01:54.239900
1000 of 2242, took: 0:02:01.938526
1100 of 2242, took: 0:02:00.558754
1200 of 2242, took: 0:02:01.527416
1300 of 2242, took: 0:02:03.597171
1400 of 2242, took: 0:01:57.408693
1500 of 2242, took: 0:01:50.927497
1600 of 2242, took: 0:01:53.930737
1700 of 2242, took: 0:01:59.048626
1800 of 2242, took: 0:02:00.222292
1900 of 2242, took: 0:02:01.995533
2000 of 2242, took: 0:02:00.186659
2100 of 2242, took: 0:01:57.449641
2200 of 2242, took: 0:01:52.052425


In [12]:
agent = list(temp_agent_sessions.keys())[0]
temp_agent_sessions[agent].columns
# len(temp_agent_sessions[agent])
# len(temp_agent_sessions[agent].loc[temp_agent_sessions[agent]['disconnection_duration_days'] < 1.0])



Index(['location_key', 'ID', 'start_connection', 'end_connection',
       'connection_time_hours', 'kWh', 'city', 'region_abbreviation',
       'provider', 'address', 'postal_code', 'district', 'subdistrict',
       'latitude', 'longitude', 'amount_of_sockets', 'parking_zone',
       'connection_duration', 'shifted_end_connection',
       'disconnection_duration', 'disconnection_duration_days',
       'connection_duration_days', 'weekday', 'kwh_grouped', 'car_type',
       'previous_location_key'],
      dtype='object')

In [31]:
total_nr_sessions = 0
total_nr_at_same = 0 
fracs = []
for a in temp_agent_sessions:
    short_sessions = temp_agent_sessions[a].loc[temp_agent_sessions[a]['disconnection_duration'] < pandas.to_timedelta('60min')]
    short_sessions = short_sessions.loc[short_sessions['disconnection_duration'] > pandas.to_timedelta('55min')]
    nr_short_sessions = len(short_sessions)
    if nr_short_sessions == 0:
        continue
    nr_at_same = len(short_sessions.loc[short_sessions['previous_location_key'] == short_sessions['location_key']])
    total_nr_sessions += nr_short_sessions
    total_nr_at_same += nr_at_same
    fracs.append(nr_at_same / (nr_short_sessions))

In [32]:
print(total_nr_sessions, total_nr_at_same, total_nr_at_same / total_nr_sessions)

3883 2466 0.6350759721864537


In [33]:
print(numpy.mean(fracs), numpy.std(fracs))

0.658719518051 0.40528841121


In [35]:
def get_connection_range(session):
    ''' This method generates the range of a charging session with the step
        size given in the bin_size_dist variable.

    Args:
        session (Series): The pandas Series from which to get the start and
            end times of the range.

    Returns:
        (DatetimeIndex): Containing Timestamps of the times between the
            start of the session and the end of the session with an
            interval between those times of the bin size of the simulation.
    '''

    range_session = pandas.date_range(session['start_connection'], session['end_connection'], freq = '20min')
    return (time for time in range_session)

def get_activity_patterns_centers(agent_sessions, centers_css):
    ''' This method calculates the activity patterns for each center of
        an agent.

    Args:
        agent_sessions (DataFrame): The sessions of the agent.
        centers_css (Dict[Tuple[float, float], Dict[str, Any]]): Centers
            (lon, lat) as keys and each center being a dictionary with the
            keys 'habit' and 'distance'. The value of the habit key is a set
            of charging stations (location keys), while the value of the
            distance key is a dictionary with the charging stations
            (location keys) as keys and their distance to the center (in
            meters) as value.

    Returns:
        activity_patterns (Dict[Tuple[float, float], List[float]]): A
            dictionary of activity patterns with the centers as keys.
            The activity patterns are lists of probabilities per bin with the
            probabilities indicating the chance of being connected to the
            center in the time interval of the bin.
    '''

    dists = {}
    if len(agent_sessions) > 0:
        for center in centers_css.keys():
            cluster_dataframes = [agent_sessions.loc[
                agent_sessions['location_key'] == cs] for cs in centers_css[center]['habit']]
            cluster_sessions = pandas.concat(cluster_dataframes)
            dists[center] = self.get_activity_pattern(cluster_sessions)
        return dists

    return {center: get_activity_pattern(pandas.DataFrame({}))
        for center in centers_css.keys()}

def get_activity_pattern(sessions):
    ''' This method calculates the activity pattern using the given sessions.

    Args:
        sessions (DataFrame): The sessions that will be used to
            create the activity pattern.

    Returns:
        (List[float]): The activity pattern being a list of probabilities
            per bin with the probabilities indicating the chance of being
            connected to the in the time interval of the bin.
    '''
    offset = pandas.to_datetime('01-01-2000', format='%d-%m-%Y')
    times_day = pandas.date_range(offset, offset + pandas.to_timedelta('23:59:59'), freq = '20min')

    if len(sessions) == 0:
        df = pandas.DataFrame({'times': times_day})
        df['0'] = pandas.Series([0] * len(times_day))
        df.set_index('times', drop = True, inplace = True)
        return [0] * len(times_day)

    sessions.all_dates = sessions.apply(lambda row: get_connection_range(row), axis = 1)
    list_all_dates = [list(dates) for dates in sessions.all_dates]
    times = []
    for dates in list_all_dates:
        for date in dates:
            times.append(date.replace(year = offset.year, month = offset.month, day = offset.day))
    df = pandas.DataFrame({'times': times})
    df.set_index('times', drop = False, inplace = True)
    df = df.groupby(pandas.TimeGrouper(freq = '20min')).count()

    if (len(df) < datetime.timedelta(days = 1) / datetime.timedelta(minutes = 20)):
        missing_indices = [time for time in times_day
            if time not in list(df.times.index)]
        missing_series = pandas.Series([0] * len(missing_indices),
            index = missing_indices)
        return df.times.append(missing_series).sort_index()
    else:
        return df.times
